In [18]:
import time

notebook_start_time = time.time()

# Set up environment

In [19]:
import sys
from pathlib import Path


def is_google_colab() -> bool:
    if "google.colab" in str(get_ipython()):
        return True
    return False


def clone_repository() -> None:
    !git clone https://github.com/decodingml/hands-on-recommender-system.git
    %cd hands-on-recommender-system/


def install_dependencies() -> None:
    !pip install --upgrade uv
    !uv pip install --all-extras --system --requirement pyproject.toml


if is_google_colab():
    clone_repository()
    install_dependencies()

    root_dir = str(Path().absolute())
    print("⛳️ Google Colab environment")
else:
    root_dir = str(Path().absolute().parent)
    print("⛳️ Local environment")

# Add the root directory to the `PYTHONPATH` to use the `recsys` Python module from the notebook.
if root_dir not in sys.path:
    print(f"Adding the following directory to the PYTHONPATH: {root_dir}")
    sys.path.append(root_dir)

⛳️ Local environment


# Inference pipeline: Deploying and testing the inference pipeline 

In this notebook, you'll deploy the inference pipeline.

## 📝 Imports

In [20]:
import warnings

warnings.filterwarnings("ignore")

from loguru import logger

from recsys import hopsworks_integration
from recsys import inference

## <span style="color:#ff5f27">🔮 Connect to Hopsworks Feature Store </span>

In [21]:
project, fs = hopsworks_integration.get_feature_store()

2024-11-21 14:51:19.428 | INFO     | recsys.hopsworks_integration.feature_store:get_feature_store:12 - Loging to Hopsworks using HOPSWORKS_API_KEY env var.


Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/15551
Connected. Call `.close()` to terminate connection gracefully.


# Deploying the ranking inference pipeline


You start by deploying your ranking model. Since it is a CatBoost model you need to implement a `Predict` class that tells Hopsworks how to load the model and how to use it.

With that in place, you can finally deploy your model.

In [22]:
ranking_deployment = hopsworks_integration.ranking_serving.HopsworksRankingModel.deploy(project=project)

Connected. Call `.close()` to terminate connection gracefully.


Uploading: 0.000%|          | 0/4306 elapsed<00:00 remaining<?

Uploading: 0.000%|          | 0/1117 elapsed<00:00 remaining<?

Deployment created, explore it at https://c.app.hopsworks.ai:443/p/15551/deployments/335876
Before making predictions, start the deployment by using `.start()`


In [23]:
ranking_deployment.start()

  0%|          | 0/6 [00:00<?, ?it/s]

Start making predictions by using `.predict()`


In [24]:
# Check logs in case of failure
ranking_deployment.get_logs(component="transformer", tail=200)

Explore all the logs and filters in the Kibana logs at https://c.app.hopsworks.ai:443/p/15551/deployments/335876

Instance name: ranking-transformer-default-00001-deployment-79f4f5fd69-kctmb
2024-11-21 12:51:56.336 7 root INFO [<module>():180] Loading serving script
2024-11-21 12:51:59.353 7 root INFO [__init__():117] Initializing transformer for deployment: ranking
Connected. Call `.close()` to terminate connection gracefully.
SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/srv/hops/hadoop-3.2.0.12-EE-RC0/share/hadoop/common/lib/log4j-slf4j-impl-2.19.0.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/srv/hops/hadoop-3.2.0.12-EE-RC0/share/hadoop/hdfs/lib/log4j-slf4j-impl-2.19.0.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
2024-11-21 12:52:09.207 7 root INFO 

## <span style="color:#ff5f27"> Test the ranking inference pipeline</span>


In [25]:
def get_top_recommendations(ranked_candidates, k=3):
    return [candidate[-1] for candidate in ranked_candidates["ranking"][:k]]

In [26]:
# Define a test input example
test_ranking_input = {
    "instances": [
        {
            "customer_id": "d327d0ad9e30085a436933dfbb7f77cf42e38447993a078ed35d93e3fd350ecf",
            "month_sin": 1.2246467991473532e-16,
            "query_emb": [
                0.214135289,
                0.571055949,
                0.330709577,
                -0.225899458,
                -0.308674961,
                -0.0115124583,
                0.0730511621,
                -0.495835781,
                0.625569344,
                -0.0438038409,
                0.263472944,
                -0.58485353,
                -0.307070434,
                0.0414443575,
                -0.321789205,
                0.966559,
            ],
            "month_cos": -1.0,
        }
    ]
}

# Test ranking deployment
ranked_candidates = ranking_deployment.predict(test_ranking_input)

# Retrieve article ids of the top recommended items
recommendations = get_top_recommendations(ranked_candidates, k=3)
recommendations

['372860002', '688105013', '525518005']

In [27]:
# Check logs in case of failure
ranking_deployment.get_logs(component="transformer", tail=200)

Explore all the logs and filters in the Kibana logs at https://c.app.hopsworks.ai:443/p/15551/deployments/335876

Instance name: ranking-transformer-default-00001-deployment-79f4f5fd69-kctmb
2024-11-21 12:52:16.541 7 opensearch INFO [log_request_success():235] POST https://rest.elastic.service.consul:9200/15551__embedding_default_project_embedding_0/_search [status:200 request:0.007s]
2024-11-21 12:52:16.549 7 opensearch INFO [log_request_success():235] POST https://rest.elastic.service.consul:9200/15551__embedding_default_project_embedding_0/_search [status:200 request:0.008s]
2024-11-21 12:52:16.556 7 opensearch INFO [log_request_success():235] POST https://rest.elastic.service.consul:9200/15551__embedding_default_project_embedding_0/_search [status:200 request:0.007s]
2024-11-21 12:52:16.564 7 opensearch INFO [log_request_success():235] POST https://rest.elastic.service.consul:9200/15551__embedding_default_project_embedding_0/_search [status:200 request:0.008s]
2024-11-21 12:52:16.8

# Deploying the query inference pipeline

In [28]:
query_model_deployment = hopsworks_integration.two_tower_serving.HopsworksQueryModel.deploy(project=project)

Connected. Call `.close()` to terminate connection gracefully.


Uploading: 0.000%|          | 0/2173 elapsed<00:00 remaining<?

Deployment created, explore it at https://c.app.hopsworks.ai:443/p/15551/deployments/335877
Before making predictions, start the deployment by using `.start()`


At this point, you have registered your deployment. To start it up you need to run:

In [29]:
query_model_deployment.start()

  0%|          | 0/6 [00:00<?, ?it/s]

Start making predictions by using `.predict()`


In [30]:
# Check logs in case of failure
query_model_deployment.get_logs(component="transformer", tail=20)

Explore all the logs and filters in the Kibana logs at https://c.app.hopsworks.ai:443/p/15551/deployments/335877

Instance name: query-transformer-default-00001-deployment-79fc9889d9-f6w68
2024-11-21 12:53:06.691 7 root INFO [<module>():180] Loading serving script
2024-11-21 12:53:09.898 7 root INFO [__init__():117] Initializing transformer for deployment: query
Connected. Call `.close()` to terminate connection gracefully.
2024-11-21 12:53:10.846 7 root INFO [<module>():196] Starting KServe server
2024-11-21 12:53:10.847 7 root INFO [register_model():187] Registering model: query
2024-11-21 12:53:10.847 7 root INFO [start():129] Setting max asyncio worker threads as 12
2024-11-21 12:53:10.848 7 root INFO [serve():139] Starting uvicorn with 1 workers
2024-11-21 12:53:10.872 7 uvicorn.error INFO [serve():84] Started server process [7]
2024-11-21 12:53:10.872 7 uvicorn.error INFO [startup():45] Waiting for application startup.
2024-11-21 12:53:10.874 7 root INFO [start():62] Starting gRP

## <span style="color:#ff5f27"> Testing the inference pipeline </span>

In [31]:
# Define a test input example
data = {
    "instances": {
        "customer_id": "d327d0ad9e30085a436933dfbb7f77cf42e38447993a078ed35d93e3fd350ecf",
        "transaction_date": "2022-11-15T12:16:25.330916",
    }
}

# Test the deployment
ranked_candidates = query_model_deployment.predict(data)

# Retrieve article ids of the top recommended items
recommendations = get_top_recommendations(ranked_candidates["predictions"], k=3)
recommendations

['660497009', '824341001', '656763001']

In [32]:
# Check logs in case of failure
query_model_deployment.get_logs(component="transformer",tail=200)

Explore all the logs and filters in the Kibana logs at https://c.app.hopsworks.ai:443/p/15551/deployments/335877

Instance name: query-transformer-default-00001-deployment-79fc9889d9-f6w68
2024-11-21 12:53:06.691 7 root INFO [<module>():180] Loading serving script
2024-11-21 12:53:09.898 7 root INFO [__init__():117] Initializing transformer for deployment: query
Connected. Call `.close()` to terminate connection gracefully.
2024-11-21 12:53:10.846 7 root INFO [<module>():196] Starting KServe server
2024-11-21 12:53:10.847 7 root INFO [register_model():187] Registering model: query
2024-11-21 12:53:10.847 7 root INFO [start():129] Setting max asyncio worker threads as 12
2024-11-21 12:53:10.848 7 root INFO [serve():139] Starting uvicorn with 1 workers
2024-11-21 12:53:10.872 7 uvicorn.error INFO [serve():84] Started server process [7]
2024-11-21 12:53:10.872 7 uvicorn.error INFO [startup():45] Waiting for application startup.
2024-11-21 12:53:10.874 7 root INFO [start():62] Starting gRP

# <span style="color:#ff5f27"> Stopping the Hopsworks deployments </span>

Stop the deployment when you're not using it.

In [33]:
# Stop the ranking model deployment
ranking_deployment.stop()

# Stop the query model deployment
query_model_deployment.stop()

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

---

In [34]:
notebook_end_time = time.time()
notebook_execution_time = notebook_end_time - notebook_start_time

logger.info(
    f"⌛️ Notebook Execution time: {notebook_execution_time:.2f} seconds ~ {notebook_execution_time / 60:.2f} minutes"
)

2024-11-21 14:53:43.063 | INFO     | __main__:<module>:4 - ⌛️ Notebook Execution time: 143.69 seconds ~ 2.39 minutes


---